In [1]:
import pandas as pd
import string
import requests
import os

In [2]:
file_name = '/Users/caged/Desktop/Mining the disclosures/RSN-MiningDisclosures15-Universe_2-4.csv'
exclude = set(string.punctuation)
def clean_names(sub):
    sub['name'] = sub['Company Name'].str.upper()
    sub['name_clean'] = sub.name.apply(lambda s: ''.join(ch for ch in s if ch not in exclude))
    sub['exchange'] = sub['Ticker Symbol'].str.split(':').str[0]
    sub['ticker'] = sub['Ticker Symbol'].str.split(':').str[1]
    return sub
tickers = pd.read_csv(file_name, usecols=['Company Name', 'Ticker Symbol'], dtype='str')
tickers = clean_names(tickers)
tickers = tickers.dropna()
tickers.head()

,Company Name,Ticker Symbol,name,name_clean,exchange,ticker
0,3D SYSTEMS CORP,NYSE:DDD,3D SYSTEMS CORP,3D SYSTEMS CORP,NYSE,DDD
1,3M Co.,NYSE:MMM,3M CO.,3M CO,NYSE,MMM
2,"A10 Networks, Inc.",NYSE:ATEN,"A10 NETWORKS, INC.",A10 NETWORKS INC,NYSE,ATEN
3,AAON INC,NasdaqGS:AAON,AAON INC,AAON INC,NasdaqGS,AAON
4,AAR CORP,NYSE:AIR,AAR CORP,AAR CORP,NYSE,AIR


In [3]:
tickers.count()

Company Name     2581
Ticker Symbol    2581
name             2581
name_clean       2581
exchange         2581
ticker           2581
dtype: int64

In [4]:
unique_tickers = tickers.ticker.unique()
len(unique_tickers)

2519

In [28]:
def look_up_company_by_tickers(list_of_tickers):
    url = "http://edgaronline.api.mashery.com/v2/companies"
    params = {
        'appkey': os.environ.get('EDGAR_KEY'),
        'primarysymbols': ','.join(list_of_tickers),
    }
    response = requests.get(url=url, params=params)
    return response

In [25]:
# sample_1 = unique_tickers[0:50]
# sample_2 = unique_tickers[50:100]
list_of_tickers = []
for x in range(250):
    tickers = unique_tickers[x*10: (x+1)*10]
    list_of_tickers.append(tickers)
len(list_of_tickers)

250

In [9]:
## EXPENSIVE - DO NOT RUN UNLESS YOU HAVE TO!!!

#good_responses = {}
#bad_responses = {}
#for i, tickers in enumerate(list_of_tickers):
#    print(i)
#    response = look_up_company_by_tickers(tickers)
#    if response.status_code == 200:
#        good_responses[i] = response
#    else:
#        bad_responses[i] = response

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [12]:
len(good_responses)

66

In [16]:
cleaned_ciks = pd.DataFrame()
for i, response in good_responses.items():
    if response.status_code == 200:
        for row in response.json()['result']['rows']:
            fields = {}
            for field in row['values']:
                fields[field['field']] = field['value']
            cleaned_ciks = pd.concat([cleaned_ciks, pd.DataFrame([fields])], ignore_index=True)
cleaned_ciks.tail()  

,cik,companyname,entityid,primaryexchange,primarysymbol,siccode,sicdescription
554,0001082554,UNITED THERAPEUTICS CORP,5683,Nasdaq Global Market,UTHR,2834,Pharmaceutical Preparations
555,0001111335,VISTEON CORP,75297,NYSE,VC,3714,Motor Vehicle Parts and Accessories
556,0001129260,"VOCERA COMMUNICATIONS, INC.",113414,NYSE,VCRA,3669,"Communications Equipment, Not Elsewhere Classi..."
557,0000839923,VODAFONE GROUP PUBLIC LTD CO,6799,Nasdaq Global Market,VOD,4812,Radiotelephone Communications
558,0001487952,"VISHAY PRECISION GROUP, INC.",824763,NYSE,VPG,3670,Electronic Components And Accessories


In [17]:
cleaned_ciks.to_csv('clean_ciks_good.csv')

In [18]:
bad_responses.keys()

dict_keys([2, 11, 19, 22, 34, 41, 44, 45, 46, 48, 49, 50, 52, 53, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 104, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 233, 234, 236, 237, 238, 240, 241, 242, 243, 245, 246, 247, 248, 249])

In [26]:
list_of_tickers[2]

array(['ADEP', 'ADBE', 'ADTN', 'ADES', 'AEIS', 'AMD', '2311', '6857',
       'ACM', 'AEGN'], dtype=object)

In [44]:
bad_responses_retry_good = {}
bad_responses_retry_bad = {}
for i, response in bad_responses.items():
    print(i)
    tickers = list_of_tickers[i]
    for ticker in tickers:
        new_response = look_up_company_by_tickers([ticker])
        if new_response.status_code == 200:
            bad_responses_retry_good[ticker] = new_response
        else:
            bad_responses_retry_bad[ticker] = new_response

2
11
19
22
34
41
44
45
46
48
49
50
52
53
56
57
58
59
60
62
63
64
65
66
67
68
69
70
71
73
75
76
78
79
80
81
82
83
84
85
88
89
90
91
92
93
94
95
97
98
99
100
101
102
104
106
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
188
189
190
191
192
193
194
196
197
198
199
201
202
203
204
205
206
207
208
209
210
213
214
215
216
218
219
220
221
222
223
224
225
226
227
228
229
230
231
233
234
236
237
238
240
241
242
243
245
246
247
248
249


In [45]:
len(bad_responses_retry_good)

1391

In [50]:
len(bad_responses_retry_bad)

449

In [51]:
cleaned_ciks_2 = pd.DataFrame()
for i, response in bad_responses_retry_good.items():
    if response.status_code == 200:
        for row in response.json()['result']['rows']:
            fields = {}
            for field in row['values']:
                fields[field['field']] = field['value']
            cleaned_ciks_2 = pd.concat([cleaned_ciks_2, pd.DataFrame([fields])], ignore_index=True)
cleaned_ciks_2.tail()  

,cik,companyname,entityid,primaryexchange,primarysymbol,siccode,sicdescription
1008,0000060667,LOWES COMPANIES INC,5395,NYSE,LOW,5211,Lumber and Other Building Materials Dealers
1009,0001624899,STERIS PLC,950898,NYSE,STE,3842,"Orthopedic, Prosthetic, and Surgical Appliance..."
1010,0000101984,UNIVERSAL ELECTRONICS INC,10978,Nasdaq Global Market,UEIC,3651,Household Audio and Video Equipment
1011,0000911971,TEEKAY CORP,5342,NYSE,TK,4412,Deep Sea Foreign Transportation of Freight
1012,0000797721,VIASAT INC,8817,Nasdaq Global Market,VSAT,3663,Radio and Television Broadcasting and Communic...


In [53]:
cleaned_ciks_2.to_csv('clean_ciks_good_2.csv')

In [54]:
len(cleaned_ciks_2)

1013

In [55]:
len(cleaned_ciks)

559

In [59]:
bad_responses_retry_bad.keys()

dict_keys(['669', '6286', '6857', '6752', '500209', '6473', '7701', '3045', 'BC8', '175', '1114', '2311', '2371', '838', '6702', '2727', 'G1A', '1999', '7731', '6302', 'A017670', '1169', '6762', 'A060980', '7752', '601607', '6724', '4543', '5347', '2317', '9914', '6728', '7240', 'S51', '6902', 'TOM2', '2238', '659', '532667', '566', '6925', '2018', 'S92', 'A005930', 'A097230', 'FI-N', '3360', '7751', '3008', '200581', 'A003600', '2384', '8036', '2323', 'A010120', 'A005490', '7741', 'O1BC', '6268', '2412', '981', '6971', 'ELI1V', '6134', '148', 'HUSQ B', '2448', '8130', 'ODPV3', '552', 'A042670', '1504', '7201', '1402', 'O2D', '7735', '2474', 'EKTA B', '6141', '631', '1882', 'GGBR4', '941', '500049', '2201', 'Z74', 'F83', '2498', 'A035420', 'MMCCORP', '700', 'ASETEK', '507685', 'PS4', '500570', '532540', '2308', '6481', 'U96', '4307', '8', '6839', 'MTELEKOM', '6845', 'WRT1V', '2324', 'A032640', '267', '2204', '5947', '8299', '6504', '7762', '6954', '1133', '317', '6770', 'A011070', '234

In [66]:
bad_responses = pd.DataFrame()
for k, response in bad_responses_retry_bad.items():
    fields = dict(
        bad_id = k,
        code = response.status_code,
        response_json = response.json(),
    )
    bad_responses = pd.concat([bad_responses, pd.DataFrame([fields])], ignore_index=True)
bad_responses.tail()

,bad_id,code,response_json
444,8069,400,{'Message': 'Parameter 'primarysymbols' has in...
445,2327,400,{'Message': 'Parameter 'primarysymbols' has in...
446,500086,400,{'Message': 'Parameter 'primarysymbols' has in...
447,9613,400,{'Message': 'Parameter 'primarysymbols' has in...
448,SUPREMEIND,400,{'Message': 'Parameter 'primarysymbols' has in...


In [68]:
bad_responses.to_csv('bad_responses.csv')